# Alexnet Implementation

## https://www.nvidia.cn/content/tesla/pdf/machine-learning/imagenet-classification-with-deep-convolutional-nn.pdf


In [ ]:
import tensorflow as tf
import numpy as np

In [6]:
nb_classes = 43 #10,1000 as per dataset

features = tf.placeholder(tf.float32,[None,227,227,3])
y = tf.placeholder(tf.float32,[None,nb_classes])

<img src="alexnet.png">

tf.nn.conv2d(
    input,
    filter,
    strides,
    padding,
    use_cudnn_on_gpu=True,
    data_format='NHWC',
    dilations=[1, 1, 1, 1],
    name=None
)

tf.nn.bias_add(
    value,
    bias,
    data_format=None,
    name=None
)

### apply local response normalization to the output.

tf.nn.local_response_normalization(
    input,
    depth_radius=5,
    bias=1,
    alpha=1,
    beta=0.5,
    name=None
)

In [7]:
def Alexnet(features,weights,biases,feature_extract=False):
    #If the image is of different size let's say 32X32X3 then it needs to be converted to 227X227X3
    #features = tf.image.resize_images(features,(227,227))
    features = tf.reshape(features,[-1,227,227,3])
    
    #1st Convolution Layer
    #conv(11, 11, 96, 4, 4, padding='VALID', name='conv1')
    conv_1 = tf.nn.conv2d(features,weights['w_1'],strides=[1,4,4,1],padding='SAME',name='conv_1')
    conv_1 = tf.nn.bias_add(conv_1,biases['b_1'])
    conv_1 = tf.nn.relu(conv_1)
    conv_1 = tf.nn.local_response_normalization(conv_1,depth_radius=2.0,bias=1.0,alpha=2e-05,beta=0.75)
    # maxpool1
    # max_pool(3, 3, 2, 2, padding='VALID', name='pool1')
    conv_1 = tf.nn.max_pool(conv_1,ksize=[1,3,3,1],strides=[1,2,2,1],padding='VALID')
    
    #2nd Convolution Layer
    # conv(5, 5, 256, 1, 1, group=2, name='conv2')
    conv_2 = tf.nn.conv2d(conv_1,weights['w_2'],strides=[1,1,1,1],padding='SAME',name='conv_2')
    conv_2 = tf.nn.bias_add(conv_2,biases['b_2'])
    conv_2 = tf.nn.relu(conv_2)
    conv_2 = tf.nn.local_response_normalization(conv_2,depth_radius=2.0,bias=1.0,alpha=2e-05,beta=0.75)
    # maxpool2
    # max_pool(3, 3, 2, 2, padding='VALID', name='pool2')
    conv_2 = tf.nn.max_pool(conv_2,ksize=[1,3,3,1],strides=[1,2,2,1],padding='VALID')
    
    #3rd Convolution Layer
    # conv(3, 3, 384, 1, 1, name='conv3')
    conv_3 = tf.nn.conv2d(conv_2,weights['w_3'],strides=[1,1,1,1],padding='SAME',name='conv_3')
    conv_3 = tf.nn.bias_add(conv_3,biases['b_3'])
    conv_3 = tf.nn.relu(conv_3)
    
    #4rd Convolution Layer
    # conv(3, 3, 384, 1, 1, group=2, name='conv4')
    conv_4 = tf.nn.conv2d(conv_3,weights['w_4'],strides=[1,1,1,1],padding='SAME',name='conv_4')
    conv_4 = tf.nn.bias_add(conv_4,biases['b_4'])
    conv_4 = tf.nn.relu(conv_4)
    
    #5rd Convolution Layer
    #conv(3, 3, 256, 1, 1, group=2, name='conv5')
    conv_5 = tf.nn.conv2d(conv_4,weights['w_5'],strides=[1,1,1,1],padding='SAME',name='conv_5')
    conv_5 = tf.nn.bias_add(conv_5,biases['b_5'])
    conv_5 = tf.nn.relu(conv_5)
    # maxpool5
    # max_pool(3, 3, 2, 2, padding='VALID', name='pool5')
   
    conv_5 = tf.nn.max_pool(conv_2,ksize=[1,3,3,1],strides=[1,2,2,1],padding='VALID')
    
    # Flatten the shape of 5th Conv layer for fully connected layer
    
    flat5 = tf.reshape(conv_5, [-1, int(np.prod(conv_5.get_shape()[1:]))])
    
    #1st Fully Connected Layer
    fc6 = tf.nn.relu(tf.matmul(flat5, weights['wfc_6']) + biases['bfc_6'])
    
    #2nd Fully Connected Layer
    fc7 = tf.nn.relu(tf.matmul(fc6, weights['wfc_7']) + biases['bfc_7'])
    
    #FOR FEATURE EXTRACTION and WEIGHT FREEZING LOGIC
    if feature_extract:
        return fc7
    
    #3rd Fully Connected Layer
    fc8 = tf.nn.relu(tf.matmul(fc7, weights['wfc_8']) + biases['bfc_8'])
    probabilities = tf.nn.softmax(fc8) #fc8 is logits
    
    return probabilities
    
    
    
    



    
    
    
    
    
    

In [9]:
weights = {
    "w_1":tf.Variable(tf.truncated_normal([11,11,3,96],stddev=1e-2),name='w_1'),
    "w_2":tf.Variable(tf.truncated_normal([5,5,96,256],stddev=1e-2),name='w_2'),
    "w_3":tf.Variable(tf.truncated_normal([3,3,256,384],stddev=1e-2),name='w_3'),
    "w_4":tf.Variable(tf.truncated_normal([3,3,384,384],stddev=1e-2),name='w_4'),
    "w_5":tf.Variable(tf.truncated_normal([3,3,384,256],stddev=1e-2),name='w_5'),
    "wfc_6":tf.Variable(tf.truncated_normal([28*28*256,4096],stddev=1e-2),name='wfc_6'),
    "wfc_7":tf.Variable(tf.truncated_normal([4096,4096],stddev=1e-2),name='wfc_7'),
    "wfc_8":tf.Variable(tf.truncated_normal([4096,nb_classes],stddev=1e-2),name='wfc_8')
    
}

In [10]:
biases = {
    "b_1":tf.Variable(tf.constant(0.0,shape=[96]),name='b_1'),
    "b_2":tf.Variable(tf.constant(0.0,shape=[256]),name='b_2'),
    "b_3":tf.Variable(tf.constant(0.0,shape=[384]),name='b_3'),
    "b_4":tf.Variable(tf.constant(0.0,shape=[384]),name='b_4'),
    "b_5":tf.Variable(tf.constant(0.0,shape=[256]),name='b_5'),
    "bfc_6":tf.Variable(tf.constant(0.0,shape=[4096]),name='bfc_6'),
    "bfc_7":tf.Variable(tf.constant(0.0,shape=[4096]),name='bfc_7'),
    "bfc_8":tf.Variable(tf.constant(0.0,shape=[nb_classes]),name='bfc_8')
    
}